### Begin hier
Begin met het runnen van deze blokken code.  
Deze zijn verantwoordelijk voor imports, setup, dependencies en andere zaken. 

In [25]:
# Imports
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
import data_generator as dgen
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras.layers import Layer
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dense, Input, Concatenate, Lambda
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.debugging.set_log_device_placement(False)
print(tf.__version__)
print(len(tf.config.list_physical_devices('GPU'))>0)

print("Succes loading Imports")

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15288756271378782890
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5738397696
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12573432263527696316
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
2.8.0
True
Succes loading Imports


In [26]:
# Functions
def get_file_name(path):
    file_name = path[path.rfind('/') + 1:]
    # Splits out the file-type designation
    file_name, file_extension = file_name.split('.')
    return file_name
    
print("Succes loading Functions")

Succes loading Functions


### Heb je al data?
Als er al trainingsdata bestaat, schrijf dan simpelweg de filename bij het betreffende blok.  
Run anders het blok waarin de data nog word gegenereerd. 

In [ ]:
"""Ik heb geen data"""
# Start here if no data has been generated.

print("Provide the path to a video file.\n"
      "  (Example: C:/Users/admin/video/video.mp4)")

vid_path = input("Provide a video path: ")
vid_name = get_file_name(vid_path)
print(f"Provided path: '{vid_path}',\nfound filename: '{vid_name}'")
print("")

generator = dgen.Generator(vid_path)
generator.generate_data()
print(f"Video has been processed and exported to {generator.file_name}__trainingsData.mp4")
print(f"Data generated and exported to {generator.file_name}_trainingsData.json")
print("")

try: 
      data_path = f"generateData_OUTPUT/{vid_name}_trainingsData.json"
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")


print("The data-table was loaded. \nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

In [27]:
"""Ik heb al trainingsdata. """
# Start here if data has already been generated.
print("Provide the filename to a file in "
      "the generateData_OUTPUT folder.\n"
      "Example: perfect_example_trainingsData")
      
data_path = input("Filename: ")
vid_name = get_file_name(data_path)
print(f"Provided path: {data_path}")

try: 
      data_path = f"generateData_OUTPUT/{vid_name}.json"
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")

print("\nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

Provide the filename to a file in the generateData_OUTPUT folder.
Example: perfect_example_trainingsData
Provided path: generateData_OUTPUT/GOPR1124_Trim_trainingsData.json

Total Table size: 
Rows: 18646, Columns: 482


In [28]:
frame_data.head()

,TS,HR,PWR,heartRateZone,0,1,2,3,4,5,...,468,469,470,471,472,473,474,475,476,477
1,505,85,50,0,"[[0.557644486427307, 0.648186445236206, -0.040...","[[0.557483077049255, 0.552068829536438, -0.076...","[[0.553551793098449, 0.5780954957008361, -0.04...","[[0.54665207862854, 0.46006208658218306, -0.05...","[[0.558870553970336, 0.5247067213058471, -0.08...","[[0.559488832950592, 0.48826456069946206, -0.0...",...,"[[0.480815052986145, 0.37191823124885504, 0.00...","[[0.49536809325218206, 0.371358275413513, 0.00...","[[0.480778813362121, 0.34924191236495905, 0.00...","[[0.46612980961799605, 0.37237301468849104, 0....","[[0.48077648878097506, 0.39444035291671703, 0....","[[0.619437217712402, 0.38546937704086304, 0.02...","[[0.6340233087539671, 0.388074874877929, 0.021...","[[0.620652437210083, 0.36350065469741805, 0.02...","[[0.6047459840774531, 0.38255375623703003, 0.0...","[[0.6183456778526301, 0.40718841552734303, 0.0..."
2,505,85,50,0,"[[0.557781159877777, 0.652138113975524, -0.040...","[[0.5576397776603691, 0.555081248283386, -0.07...","[[0.5538336634635921, 0.5809364914894101, -0.0...","[[0.547837734222412, 0.46182113885879505, -0.0...","[[0.5592626929283141, 0.527342796325683, -0.08...","[[0.56026405096054, 0.490500211715698, -0.0747...",...,"[[0.48186632990837003, 0.372595608234405, 0.00...","[[0.496729910373687, 0.37259367108345004, 0.00...","[[0.48208233714103704, 0.349857658147811, 0.00...","[[0.466910362243652, 0.37237179279327304, 0.00...","[[0.481569498777389, 0.39509481191635104, 0.00...","[[0.619723141193389, 0.38572886586189203, 0.02...","[[0.6348772048950191, 0.38816174864768904, 0.0...","[[0.620543837547302, 0.363969087600708, 0.0203...","[[0.6045479774475091, 0.38323295116424505, 0.0...","[[0.6189938783645631, 0.40755006670951804, 0.0..."
3,505,85,50,0,"[[0.5626065135002131, 0.656960010528564, -0.04...","[[0.5579067468643181, 0.556771099567413, -0.07...","[[0.554864883422851, 0.582704246044158, -0.041...","[[0.5476696491241451, 0.463245302438735, -0.05...","[[0.559209465980529, 0.528769314289093, -0.080...","[[0.560060918331146, 0.49175029993057207, -0.0...",...,"[[0.482597649097442, 0.37554320693016, 0.00381...","[[0.49783867597579906, 0.37549942731857305, 0....","[[0.482793241739273, 0.35240229964256203, 0.00...","[[0.46731585264205905, 0.37538290023803705, 0....","[[0.48236334323883, 0.39839088916778503, 0.003...","[[0.6212606430053711, 0.387251198291778, 0.020...","[[0.635286450386047, 0.39012062549591003, 0.02...","[[0.6227678656578061, 0.365785002708435, 0.020...","[[0.607255220413208, 0.38423293828964206, 0.02...","[[0.619867324829101, 0.40856561064720104, 0.02..."
4,505,85,50,0,"[[0.561814308166503, 0.6575654745101921, -0.04...","[[0.559345841407775, 0.5556150674819941, -0.07...","[[0.555837929248809, 0.58216106891632, -0.0412...","[[0.548979759216308, 0.46244707703590304, -0.0...","[[0.560742497444152, 0.527681291103363, -0.080...","[[0.5615682601928711, 0.490862190723419, -0.07...",...,"[[0.48320049047470004, 0.37472867965698203, 0....","[[0.49830114841461104, 0.37439942359924305, 0....","[[0.48332807421684204, 0.35114678740501404, 0....","[[0.46801066398620605, 0.37484848499298, 0.005...","[[0.48302134871482805, 0.39804667234420704, 0....","[[0.621431291103363, 0.387765794992446, 0.0204...","[[0.635533690452575, 0.39082396030426003, 0.02...","[[0.62286365032196, 0.366147100925445, 0.02042...","[[0.60738605260849, 0.38455718755722, 0.020420...","[[0.620088934898376, 0.40927729010581904, 0.02..."
5,505,85,50,0,"[[0.5678303241729731, 0.663326919078826, -0.03...","[[0.5601344108581541, 0.554888725280761, -0.07...","[[0.556598603725433, 0.581692636013031, -0.040...","[[0.5491563081741331, 0.460878789424896, -0.05...","[[0.5613942146301271, 0.526626706123352, -0.08...","[[0.561959207057952, 0.489454329013824, -0.074...",...,"[[0.48083800077438305, 0.37290006875991805, 0....","[[0.495698124170303, 0.372656464576721, 0.0036...","[[0.48110470175743103, 0.348926812410354, 0.00...","[[0.46586400270462003, 0.3729018

In [29]:
Xtemp = frame_data.drop(columns=['TS', 'HR', 'PWR', 'heartRateZone'])
Y = frame_data["heartRateZone"]
X = []
temp = []



for index, row in Xtemp.iterrows():
    for i in range(0,478):
        for g in range(0,3):
            temp.append(row[i][0][g])
    X.append(temp)
    temp = []
Y.to_numpy()
Y = Y.values.reshape(-1,1)
temps = np.zeros((len(Y), 6))

for i in range(0,len(Y)):
    if Y[i][0] == 0:
        temps[i][0] = 1
    elif Y[i][0] == 1:
        temps[i][1] = 1
    elif Y[i][0] == 2:
        temps[i][2] = 1
    elif Y[i][0] == 3:
        temps[i][3] = 1
    elif Y[i][0] == 4:
        temps[i][4] = 1
    elif Y[i][0] == 5:
        temps[i][5] = 1

print(temps)


[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


In [30]:
#tf.debugging.set_log_device_placement(False)

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, temps, test_size=0.33, random_state=42)

X_train = np.array(X_train)

print(X_train.shape)
print(Y_train.shape)


(12492, 1434)
(12492, 6)


In [32]:
inputTensor = Input(shape=(1434,))
layers = []

for i in range(0, 1434, 3):
    layers.append(Lambda(lambda x: x[:,i: i+3], output_shape=((3,)))(inputTensor))
for i in range(0, 478):
    layers[i] = Dense(1)(layers[i])
coordslayer = Concatenate()(layers)

hidden1 = Dense(477*2, activation='relu')(coordslayer)
hidden2 = Dense(477, activation='relu')(hidden1)
hidden3 = Dense(240, activation='relu')(hidden2)
hidden4 = Dense(100, activation='relu')(hidden3)
hidden5 = Dense(20, activation='relu')(hidden4)

outputTensor = Dense(6, activation='sigmoid')(hidden5)
model = Model(inputs=inputTensor, outputs=outputTensor)

In [24]:
keras.backend.clear_session()

In [ ]:
print(model.summary())

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(X_train, Y_train, epochs=150, batch_size=1000)

Epoch 1/150
125/125 [==============================] - 25s 98ms/step - loss: 1.1200 - accuracy: 0.5309
Epoch 2/150
125/125 [==============================] - 12s 93ms/step - loss: 0.9368 - accuracy: 0.5857
Epoch 3/150
125/125 [==============================] - 12s 93ms/step - loss: 0.8992 - accuracy: 0.6028
Epoch 4/150
125/125 [==============================] - 12s 95ms/step - loss: 0.8863 - accuracy: 0.6046
Epoch 5/150
125/125 [==============================] - 14s 108ms/step - loss: 0.8761 - accuracy: 0.6102
Epoch 6/150
125/125 [==============================] - 13s 102ms/step - loss: 0.8851 - accuracy: 0.6118
Epoch 7/150
125/125 [==============================] - 12s 96ms/step - loss: 0.8719 - accuracy: 0.6141
Epoch 8/150
125/125 [==============================] - 12s 96ms/step - loss: 0.8616 - accuracy: 0.6181
Epoch 9/150
125/125 [==============================] - 13s 100ms/step - loss: 0.8696 - accuracy: 0.6171
Epoch 10/150
125/125 [==============================] - 12s 98ms/step 

Epoch 1/150
125/125 [==============================] - 13s 104ms/step - loss: 0.6258 - accuracy: 0.7290
Epoch 2/150
125/125 [==============================] - 13s 103ms/step - loss: 0.6269 - accuracy: 0.7327
Epoch 3/150
125/125 [==============================] - 12s 100ms/step - loss: 0.6225 - accuracy: 0.7362
Epoch 4/150
125/125 [==============================] - 12s 98ms/step - loss: 0.6246 - accuracy: 0.7320
Epoch 5/150
125/125 [==============================] - 12s 98ms/step - loss: 0.6211 - accuracy: 0.7331
Epoch 6/150
125/125 [==============================] - 12s 100ms/step - loss: 0.6241 - accuracy: 0.7311
Epoch 7/150
125/125 [==============================] - 13s 100ms/step - loss: 0.6203 - accuracy: 0.7339
Epoch 8/150
125/125 [==============================] - 12s 99ms/step - loss: 0.6276 - accuracy: 0.7308
Epoch 9/150
125/125 [==============================] - 13s 103ms/step - loss: 0.6177 - accuracy: 0.7378
Epoch 10/150
125/125 [==============================] - 13s 103ms/s

Epoch 1/150
13/13 [==============================] - 1s 104ms/step - loss: 0.5269 - accuracy: 0.7742
Epoch 2/150
13/13 [==============================] - 1s 101ms/step - loss: 0.5106 - accuracy: 0.7827
Epoch 3/150
13/13 [==============================] - 1s 101ms/step - loss: 0.5051 - accuracy: 0.7846
Epoch 4/150
13/13 [==============================] - 1s 101ms/step - loss: 0.5020 - accuracy: 0.7863
Epoch 5/150
13/13 [==============================] - 1s 101ms/step - loss: 0.5013 - accuracy: 0.7847
Epoch 6/150
13/13 [==============================] - 1s 104ms/step - loss: 0.4989 - accuracy: 0.7874
Epoch 7/150
13/13 [==============================] - 1s 102ms/step - loss: 0.4968 - accuracy: 0.7900
Epoch 8/150
13/13 [==============================] - 1s 102ms/step - loss: 0.4968 - accuracy: 0.7883
Epoch 9/150
13/13 [==============================] - 1s 102ms/step - loss: 0.4952 - accuracy: 0.7883
Epoch 10/150
13/13 [==============================] - 1s 103ms/step - loss: 0.4945 - accura

In [38]:
# evaluate the keras model
X_test = np.array(X_test)
accuracy = model.evaluate(X_test, Y_test)
print(accuracy)

193/193 [==============================] - 8s 41ms/step - loss: 0.8036 - accuracy: 0.7171
[0.8036181330680847, 0.7170946002006531]


In [44]:
model.save('models/model_0.8_0.72')

INFO:tensorflow:Assets written to: models/model_0.8_0.72\assets


In [45]:
model = keras.models.load_model('models/model_0.8_0.72')